Connect to google drive(for downloading photos)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os

path = "/content/gdrive/MyDrive/photos"
# Check if file exists
if os.path.isfile(path):
  os.mkdir(path)

Configure ngrok

In [ ]:
import os
!pip install pyngrok
from pyngrok import ngrok

port = 5000

ngrok_token =  os.getenv('NGROK_TOKEN')

#Setting an auth token allows us to open multiple tunnels at the same time
ngrok.set_auth_token(ngrok_token)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

FLASK(download photos or sending echo message)

In [ ]:
import requests
from flask import Flask, request
import threading

app = Flask(__name__)

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = "https" + public_url.split('http')[1]
print(app.config["BASE_URL"])

token = os.getenv('BOT_TOKEN')
bot = Bot(token, app.config["BASE_URL"])

@app.route("/", methods=["GET", "POST"])
def receive_update():
    if request.method == "POST":
        print(request.json)
        chat_id = request.json["message"]["chat"]["id"]
        photos = request.json["message"].get("photo")
        if photos:
          photo = photos[-1]
          photo_path, downloaded_photo = bot.get_photo(photo['file_id'])
          print(downloaded_photo)
          downloaded_photo.save('/content/gdrive/MyDrive/' + photo_path)

          # Machine Learning stuff
          #<ML CODE>

          # Delete photo after processing
          # os.remove('/content/gdrive/MyDrive/' + photo_path)
        else:
          # Echo Bot
            print(request.json["message"]["text"])
            bot.send_message(chat_id, request.json["message"]["text"])
    return request.json["message"]

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()
